In [112]:
#importing the first sql libary for py withoout math functionality; but credentials adoptions 
import ibm_db

In [113]:
#The database credentials to teh db2 warehouse to access the tables.
#These credentials are only for an instance in the cloud account, that allows one instance per session

dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = "BLUDB"

dsn_hostname = "dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net"
dsn_port = "50000"
dsn_protocol = "TCPIP"   

dsn_uid = "bfp69501"
dsn_pwd= "xr2wcxn5tzkdt@lp"


In [114]:
dsn = (
    "DRIVER = {0};"
    "DATABASE = {1};"
    "HOSTNAME = {2};"
    "PORT= {3};"
    "PROTOCOL= {4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)


In [115]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Unable to connect:  [IBM][CLI Driver] SQL0438N  Application raised error or warning with diagnostic text: "Exceeded maximum limit of 5 connections. Connection refused".  SQLSTATE=42502 SQLCODE=-438


In [151]:
#NB this API may take a while before it connects an instance to the IBM DB2 free services
#Please run the code multiple times, if you are prompted for with a warning for expired credentials do not hesitate to contact me 



In [117]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)
print(client)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB


In [118]:
#import libraries for math and plots; establish a different ibm API to retrieve and show data contained
import pandas
import numpy
import scipy 
from scipy import stats
import ibm_db_dbi
import matplotlib 
%matplotlib inline 

In [119]:
#The code below will now show the data that we are working with; that is stored and structured in tables in DB2

#connection for pandas
pconn = ibm_db_dbi.Connection(conn)
instance1= "select * from CPI" 
instance2 = "select * from ECOINDI"
instance3 = "select * from CRISIS" 
show1 = pandas.read_sql(instance1, pconn)   #this data is now in pandas dataframe 
show2 = pandas.read_sql(instance2, pconn)
show3 = pandas.read_sql(instance3, pconn)

In [120]:

print("Consumer Price Index") 
CPI = show1.drop(["Consumer_Price_Index___February_2019_100_", "Column_2", "Column_3", "Column_4", "Column_5", "Column_6","Column_7", "Column_8", "Column_9", "Column_10", "Column_11", "Column_12", "Column_13"], axis = 1)
CPI.columns=['CPI_mt', 'Prd_All_Items' , 'Inflation_Rate_Percent_Monthly', 'Inflation_Rate_Percent_Annual']
CPi= CPI.replace('None', '').astype(str)
CPi.head(5)


Consumer Price Index


,CPI_mt,Prd_All_Items,Inflation_Rate_Percent_Monthly,Inflation_Rate_Percent_Annual
0,None,None,None,None
1,None,All Items,Inflation Rate (%) Monthly,Inflation Rate (%) Annual
2,None,100,None,None
3,None,None,None,None
4,2009,"56,2",None,None


In [157]:

print("Macro-economic Indicators")
show2= show2.replace('None', '').astype(str)
show2.columns=['Indicator', '2009' , '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
show2.head(8)

Macro-economic Indicators


,Indicator,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,,,,,,,,,,,
1,,,,,,,,,,,
2,,,,,,,,,,,
3,,,,,,,,,,,
4,,,,,,,,,,,
5,REAL SECTOR,,,,,,,,,,
6,Real GDP at 2012 Market prices (US$' m),10 735,12 847,"14 670,00","17 114,80","17 455,30","17 870,20","18 188,30","18 325,80","19 187,80",
7,Nominal GDP at Market (US$'m up to 2016),9 666,12 042,"14 101,80","17 114,80","19 091,00","19 495,50","19 963,10","20 548,70","22 813,00",


In [153]:
#Do not drop all countries before comparison we might nedd the data 
#juy = range(969)
#show4= show3.drop(index=juy, axis= 0)

print("Global economic crisis")
show4 = show3.drop(["inflation_crises", "case", "banking_crisis", "currency_crises", "domestic_debt_in_default", "sovereign_external_debt_default", "gdp_weighted_default","independence"], axis = 1)
show4= show4.replace('None', '').astype(str)
show4.head(5)
#refined global economic crisis data



Global economic crisis


,cc3,country,year,systemic_crisis,exch_usd,inflation_annual_cpi
0,DZA,Algeria,1870,1,0.052264,3.441455696
1,DZA,Algeria,1871,0,0.052798,14.14913958
2,DZA,Algeria,1872,0,0.052274,-3.718592965
3,DZA,Algeria,1873,0,0.05168,11.20389701
4,DZA,Algeria,1874,0,0.051308,-3.848560701


In [144]:
CPi.dtypes


CPI_mt                            object
Prd_All_Items                     object
Inflation_Rate_Percent_Monthly    object
Inflation_Rate_Percent_Annual     object
dtype: object

In [124]:

 show2.dtypes


Indicator    object
2009         object
2010         object
2011         object
2012         object
2013         object
2014         object
2015         object
2016         object
2017         object
2018         object
dtype: object

In [125]:

show4.dtypes

cc3                     object
country                 object
year                    object
systemic_crisis         object
exch_usd                object
inflation_annual_cpi    object
dtype: object

In [126]:
#modification of the cpi and defination of attributes

df_CPi = cpi.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

df_CPi = df_cpi.replace("None", "").astype(str)

df_CPi['CPI_mt'] = pandas.to_numeric(df_cpi['CPI_mt'], errors='coerce')

df_CPi['Prd_All_Items'] = pandas.to_numeric(df_cpi['Prd_All_Items'], errors='coerce')

df_CPi['Inflation_Rate_Percent_Monthly'] = pandas.to_numeric(df_cpi['Inflation_Rate_Percent_Monthly'], errors='coerce')

df_CPi['Inflation_Rate_Percent_Annual'] = pandas.to_numeric(df_cpi['Inflation_Rate_Percent_Annual'], errors= 'coerce')

df_CPi.dtypes


CPI_mt                            float64
Prd_All_Items                     float64
Inflation_Rate_Percent_Monthly    float64
Inflation_Rate_Percent_Annual     float64
dtype: object

In [149]:
# modification of crisis modification of attributes 
#let all pythin programmers take time to acknowledge the programmer who invented the pandas module i been stressing the whole day!!!!

show4['exch_usd'] = pandas.to_numeric(show4['exch_usd'], errors='coerce')

show4['inflation_annual_cpi'] = pandas.to_numeric(show4['inflation_annual_cpi'], errors='coerce')

show4['year'] = pandas.to_datetime(show4['year'], errors='coerce')


show4.dtypes

cc3                             object
country                         object
year                    datetime64[ns]
systemic_crisis                 object
exch_usd                       float64
inflation_annual_cpi           float64
dtype: object

In [133]:
#essentials establieshed on Consumer Price Index Data and global crisi data  Now its time for some real Maths :)

#Import magic sql environment to view tables for plots

!pip install ipython-sql


In [134]:
%load_ext sql

%sql ibm_db_sa://bfp69501:xr2wcxn5tzkdt%40lp@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB


try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to Tinashe's Db2: ", dsn_database, "As user: ", dsn_uid, "on Host: ", dsn_hostname)

except:
    print ("Unable to connect, Logon Instance can not exceed 5! Try Again a bit later/ Read Error and contact Tinashe for Help :)", ibm_db.conn_errormsg() )
    


The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Unable to connect, Logon Instance can not exceed 5! Try Again a bit later/ Read Error and contact Tinashe for Help :) [IBM][CLI Driver] SQL0438N  Application raised error or warning with diagnostic text: "Exceeded maximum limit of 5 connections. Connection refused".  SQLSTATE=42502 SQLCODE=-438


In [143]:

gk= %sql  SELECT * FROM ECOINDI
gk
#this SQL code will hold the dataframe in a scaled tabular view to show other indicators that were in the dataset grouped per industrial sector


 * ibm_db_sa://bfp69501:***@dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net:50000/BLUDB
Done.


Indicator,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
None,None,None,None,None,None,None,None,None,None,None
None,None,None,None,None,None,None,None,None,None,None
None,None,None,None,None,None,None,None,None,None,None
None,None,None,None,None,None,None,None,None,None,None
None,None,None,None,None,None,None,None,None,None,None
REAL SECTOR,None,None,None,None,None,None,None,None,None,None
Real GDP at 2012 Market prices (US$' m),10 735,12 847,"14 670,00","17 114,80","17 455,30","17 870,20","18 188,30","18 325,80","19 187,80",None
Nominal GDP at Market (US$'m up to 2016),9 666,12 042,"14 101,80","17 114,80","19 091,00","19 495,50","19 963,10","20 548,70","22 813,00",None
GDP at Market Prices % changes,"5,4","19,7","14,2","16,7",2,"2,4","1,8","0,8","4,7",None
GDP per capita,"790,3","976,1","1 131,80","1 310,40","1 428,00","1 428,00","1 431,70","1 440,80","1 510,80",None


In [158]:
#LIST OF USEFUL INDICATORS FOR ANALYSIS 
Indicators=['GDP per capita', 'Export growth (%)', 'Import growth (%)', 'Trade Balance (US$M)', 'POPULATION (millions)']

ecoIndi= pandas.DataFrame(show2, columns = ['Indicator', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], index=[9, 27, 28, 87])
ecoIndi

#Nice:) 

,Indicator,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
9,GDP per capita,"790,3","976,1","1 131,80","1 310,40","1 428,00","1 428,00","1 431,70","1 440,80","1 510,80",
27,Export growth (%),"-0,90%","107,10%","40,40%","-11,80%","-4,00%","-3,50%","-2,90%","2,40%","18,00%","11,70%"
28,Import growth (%),"22,20%","87,80%","46,70%","-11,30%","1,40%","-7,00%","-4,10%","-13,70%","5,80%","28,50%"
87,POPULATION (millions),"12,2","12,3","12,5","13,1","13,4","13,7","13,9","14,3","14,6","14,9"


In [159]:
#remember now we are working with 
#ecoIndi show4 as crisis CPi as CPi

ecoIndi.describe()

,Indicator,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,4,4,4,4,4,4,4,4,4,4,4
unique,4,4,4,4,4,4,4,4,4,4,4
top,Export growth (%),"-0,90%","12,3","12,5","-11,30%","1,40%","-7,00%","-4,10%","14,3","1 510,80","14,9"
freq,1,1,1,1,1,1,1,1,1,1,1


In [160]:
CPi.describe()  #not useful

,CPI_mt,Prd_All_Items,Inflation_Rate_Percent_Monthly,Inflation_Rate_Percent_Annual
count,152,152,152,152
unique,25,66,35,74
top,None,None,None,None
freq,16,14,26,28


In [161]:
#to be honest Zimbabwe needs more data assets this is useless 
show4.describe() #not useful

,cc3,country,year,systemic_crisis,exch_usd,inflation_annual_cpi
count,1059,1059,1059,1059,1059,1059
unique,13,13,155,2,773,1022
top,EGY,Egypt,1996,0,0,0.0
freq,155,155,13,977,53,17


In [162]:

#income_vs_hardship = %sql SELECT FROM ECOINDI;


#plot = sns.jointplot(x='per_capita_income_',y='hardship_index', data=income_vs_hardship.DataFrame())



#Pending developments on this Notebook through GitHub
#WebScrapping In Progress for improvements on Dataset used in this project
#Used data was Free from Kuggle/ RBZ (already Refined)/ 


ibm_db.close(conn)

True